In [1]:
import numpy as np
# import tensorflow as tf
# import keras
# from keras.models import load_model
# import os
import time
import os
#from multiprocessing import Pool
import matplotlib.pyplot as plt
%matplotlib inline

# from keras.optimizers import Adam

# from fully_conv_model_for_lidar import fcn_model
from util_func import *
# from train import *

# from keras.utils.generic_utils import get_custom_objects
# #loss = SSD_Loss(neg_pos_ratio=neg_pos_ratio, alpha=alpha)
# get_custom_objects().update({"my_loss": my_loss})


In [2]:
def list_of_data(data_dir):
    list_of_lidar = []
    list_of_gtbox = []
    for f in os.listdir(data_dir):
        path = os.path.join(data_dir, f)
        lidar_path = os.path.join(path, 'lidar')
        gtbox_path = os.path.join(path, 'gt_boxes3d')
        num_files = len(os.listdir(lidar_path))

        lidar = [os.path.join(lidar_path, 'lidar_'+str(i)+'.npy') for i in range(num_files) ]
        gtbox = [os.path.join(gtbox_path, 'gt_boxes3d_'+str(i)+'.npy') for i in range(num_files) ]
        list_of_lidar += lidar
        list_of_gtbox += gtbox
    return list_of_lidar, list_of_gtbox

In [3]:
data_dir = './all_data/extract_kiti/'
list_of_lidar, list_of_gtbox = list_of_data(data_dir)
print(len(list_of_lidar))

12482


In [4]:
num_cars = np.zeros(len(list_of_gtbox))
for i in range(len(list_of_gtbox)):
    gt_box = np.load(list_of_gtbox[i])
    num_cars[i] = len(gt_box)

In [11]:
unique, counts = np.unique(num_cars, return_counts=True)
dict(zip(unique, counts))

{0.0: 3723,
 1.0: 2086,
 2.0: 1408,
 3.0: 1020,
 4.0: 1003,
 5.0: 913,
 6.0: 697,
 7.0: 463,
 8.0: 318,
 9.0: 241,
 10.0: 170,
 11.0: 117,
 12.0: 103,
 13.0: 79,
 14.0: 86,
 15.0: 44,
 16.0: 4,
 17.0: 2,
 18.0: 4,
 19.0: 1}

In [5]:
indx = {}
for i in range(len(num_cars)):
    if num_cars[i] not in indx.keys():
        indx[num_cars[i]] = [i]
    else:
        indx[num_cars[i]].append(i)
print(indx.keys())

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0]


In [6]:
import pickle

with open('./saved_model/numcar_ind.pickle', 'wb') as f:
    pickle.dump(indx, f)

with open('./saved_model/numcar_ind.pickle', 'rb') as f:
    ind_load = pickle.load(f)

print(indx == ind_load)


True


In [8]:
print(int(0.1*len(ind_load[0])))

372


In [11]:
def undersample_data(list_of_lidar, list_of_gtbox, pickle_index, percent_noncar = 0.1):
    with open(pickle_index, 'rb') as f:
        ind_load = pickle.load(f)
    size_noncar = int(0.1*len(ind_load[0]))
    ind_0 = list(np.random.choice(ind_load[0], size=size_noncar, replace=False))
    for i in range(1, len(ind_load)):
        ind_0 += ind_load[i]
    new_list_of_lidar = [list_of_lidar[i] for i in ind_0]
    new_list_of_gtbox = [list_of_gtbox[i] for i in ind_0]
   
    return new_list_of_lidar, new_list_of_gtbox

pickle_index = './saved_model/numcar_ind.pickle'
new_list_of_lidar, new_list_of_gtbox = undersample_data(list_of_lidar, list_of_gtbox, 
                                                       pickle_index, percent_noncar = 0.1)
print(len(new_list_of_lidar), len(new_list_of_gtbox))

(9131, 9131)


In [24]:
for i in indx.keys():
    print(i,' : ',np.random.choice(indx[i]), len(indx[i]) )

(0.0, ' : ', 10142, 3723)
(1.0, ' : ', 7244, 2086)
(2.0, ' : ', 1249, 1408)
(3.0, ' : ', 8034, 1020)
(4.0, ' : ', 7516, 1003)
(5.0, ' : ', 1465, 913)
(6.0, ' : ', 3564, 697)
(7.0, ' : ', 3042, 463)
(8.0, ' : ', 896, 318)
(9.0, ' : ', 4277, 241)
(10.0, ' : ', 11149, 170)
(11.0, ' : ', 11074, 117)
(12.0, ' : ', 11246, 103)
(13.0, ' : ', 12287, 79)
(14.0, ' : ', 12381, 86)
(15.0, ' : ', 12406, 44)
(16.0, ' : ', 12370, 4)
(17.0, ' : ', 677, 2)
(18.0, ' : ', 681, 4)
(19.0, ' : ', 680, 1)
